Patch Daddy: 
    

In [ ]:
#imports, load feature df, set directory and colours

%load_ext autoreload
%autoreload 2


from utils.helper_functions import loopCombinations, loopCombinations_stats
from utils.plotter import drug_aplication_visualisation, plot_all_FI_curves, plot_FI_AP_curves
from utils import igor_utils 
from ephys import ap_functions
import openpyxl
import pandas as pd
import numpy as np
import os
import warnings

#DJ?
# os.environ["OMP_NUM_THREADS"] ='3'
# warnings.filterwarnings('ignore')
   
ROOT = os.getcwd() #This gives terminal location (terminal working dir)
INPUT_DIR = f'{ROOT}/input'
OUTPUT_DIR = f'{ROOT}/output'

feature_df = pd.read_excel (f'{INPUT_DIR}/feature_df_py.xlsx') 

data_path = f'{INPUT_DIR}/PatchData/' #THIS IS HARD CODED INTO make_path(file_folder)

color_dict = {"PRE":"black", "CONTROL": 'grey', "TCB2":'green', "DMT":"teal", "PSIL":"orange", "LSD":"purple", "MDL":'blue'} 

In [ ]:
#Trace Plotting
drug_aplication_visualisation(feature_df, color_dict) # generates PDF of drug aplications # in plotters in utils

plot_all_FI_curves(feature_df,  color_dict)  # generates PDF with all FI curves for single cell labed with drug and aplication order #### MAKE HZ NOT APs per sweep also isnt it in pA not nA??

plot_FI_AP_curves(feature_df) #generated PDF with FI-AP for each cell
